### 1. Policy Evaluation

#### Rappel

L'algorithme de Policy Evaluation calcule $v_\pi$ pour $\pi$, pour chaque state $s$. $\pi$ étant la policy d'un agent, son comportement, et $v_\pi$ étant une fonction, un tableau en réalité, associant à chaque state $s$ le return espéré en suivant $\pi$ et en étant dans le state $s$. Ce return équivaut à la somme des futurs rewards reçus à partir du state $s$.

Ainsi, l'algorithme applique itérativement la première équation d'espérance de Bellman. Voici son update:

\begin{align}
V(s) \; & \leftarrow \sum_{a} \pi(a|s) \sum_{s'} p(s'|s,a) \Big[r(s,a,s') + γ V(s') \Big], \; \forall s
\end{align}

Qu'on peut aussi écrire :

\begin{align}
V(s) \; & \leftarrow \sum_{a} \pi(a|s) Q(s,a), \; \forall s
\end{align}

___

<img src="./PE.png" width="550" height="200" align="center"/>

___

#### Environnement

Implémentation du Row Gridworld

In [8]:
class GridworldRow():
    def __init__(self, size=5):
        self.size = size

        self.nS = size
        self.nA = 2

        self.MAX_X = size-1

        P = {}
        
        for s in range(self.nS):                
            dynamics_s = {}
            
            for a in range(self.nA):
                s_prime_list = []

                p = 1 if s != 0 and s != self.nS-1 else 0

                if(a == 0):
                    s_prime = max(0, s-1)
                else:
                    s_prime = min(self.MAX_X, s+1)

                if (s_prime == 0):
                    reward = -100
                    done = True
                elif(s_prime == self.MAX_X):
                    reward = 10
                    done = True
                else:
                    reward = 0
                    done = False

                s_prime_list.append((p, s_prime, reward, done))
                dynamics_s.update({a:s_prime_list})

            P.update({s: dynamics_s})
            
        self.P = P  

#### Algorithme

In [9]:
import numpy as np

In [10]:
env = GridworldRow()

In [11]:
pi = np.ones([env.nS, env.nA]) * 0.5
V = np.zeros([env.nS, 1])

gamma = 0.99 #facteur de remise du return
theta = 0.00001 #seuil de similitude requis pour stopper les updates

Fonction qui calcule $q_\pi(s,a)$ grâce à $V$. Renvoit donc $Q(s,a) = \sum_{s'}p(s'|s,a) \Big[r(s,a,s') + \gamma V(s') \Big]$

In [12]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    
    for (p_sPrime, sPrime, r_ss_a, done) in env.P[s][a]:
        q += p_sPrime * (r_ss_a + gamma * V[sPrime])
        
    return q

Boucles d'updates :

In [13]:
i=0
while True:
    i+=1
    delta = 0
    
    for s in range(env.nS):
        V_new = 0

        for a in range(env.nA):
            prob_a = pi[s][a]
            q_s_a = compute_q_value_for_s_a(env, V, s, a, gamma)

            V_new += prob_a * q_s_a

        delta = max(delta, np.abs(V_new - V[s]))
        V[s] = V_new

    if(delta < theta) :
        print("Terminé après " + str(i) + " itérations")
        break

Terminé après 22 itérations


In [14]:
V

array([[  0.        ],
       [-71.62196676],
       [-43.6807471 ],
       [-16.62196981],
       [  0.        ]])